# Implementing and fitting a simple syrinx model

Model based on [Mindlin et al., 2003](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.68.041908). 

We want to fit the system of first-order nonlinear equations:

$$
\dot{x} = y \\
\dot{y} = -\epsilon x - C x^2 y + By - D_0
$$
where
$$
\epsilon = \epsilon_1 + \epsilon_2 K(t) \\
B = \beta_1 + \beta_2 P(t) \\
D_0 = \delta D(t)
$$
and $K(t)$, $D(t)$, and $P(t)$ are the (linear envelopes of) tension in the ventral syringeal muscle (vS), the tracheobronchialis dorsalis (dTB), and sub-syringeal air pressure, respectively.

From the original paper, we take parameter values
$$
\epsilon_1 = 1.25 \times 10^8 \, \mathrm{s}^{-2} \\
\epsilon_2 = 7.5 \times 10^9 \, \mathrm{V}^{-1}\cdot \mathrm{s}^{-2} \\
C = 2 \times 10^8 \, \mathrm{cm}^{-2} \cdot \mathrm{s}^{-1} \\
\beta_1 = -2 \times 10^3 \, \mathrm{s}^{-1} \\
\beta_2 = 5.3 \times 10^4 \, \mathrm{V}^{-1}\cdot \mathrm{s}^{-1} \\
\delta = 15 \times 10^6 \, \mathrm{cm}\cdot\mathrm{V}^{-1} \cdot \mathrm{s}^{-2}
$$

But whereas the original paper used measured values for $K$, $P$, and $D$, we use simpler time series constructed to have the same shape:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [ ]:
sr = 4e4  # data sampling rate (Hz)
t_axis = np.arange(0, 2, 1/sr)

In [ ]:
def make_cos_pulse(freq, amp, cutoff, phase, dc):
    fn = lambda t: amp * np.maximum(np.sin(2 * np.pi * freq * t + phase), cutoff) + dc 
    return fn

pfreq = 2  # pressure pulse frequency (Hz)
pA = 0.025  # pressure amplitude (Volts)
p0 = 0.01  # pressure DC offset (Volts)
pcutoff = -.7
pphase = -3 * np.pi/4
ppulse = make_cos_pulse(pfreq, pA, pcutoff, pphase, p0)

P = ppulse(t_axis)
plt.plot(t_axis, P)

In [ ]:
def make_tension_pulse_fn(loc, shape=1, scale=1, peak=1):
    norm = peak * np.exp(shape - shape * np.log(shape) - shape * np.log(scale)) 

    fn = lambda t: norm * np.exp((t - loc)/scale) * np.maximum((loc - t), 0)**shape 

    return fn
    
        
kshape = 5  # shape parameter of gamma function
kscale = 0.025  # rate parameter of gamma function (s)
kloc = 1  # location of peak
kpeak = 0.06  # peak value (Volts)

from functools import partial

pulse = partial(make_tension_pulse_fn, shape=kshape, scale=kscale, peak=kpeak)
K = pulse(0.5)(t_axis) + pulse(1)(t_axis) + pulse(1.5)(t_axis)


In [ ]:
dfreq = 2 
dlocs1 = np.arange(0.1, t_axis[-1], 1/dfreq)  # pressure pulse frequency (Hz)
dlocs2 = np.arange(0.45, t_axis[-1], 1/dfreq)  # pressure pulse frequency (Hz)
dlocs = np.sort(np.concatenate([dlocs1, dlocs2]))
dA = 0.05
dwid = 0.01

gpulse = lambda t, loc=0: dA * np.exp(-0.5 * (t - loc)**2/dwid**2)


D = np.sum(np.stack([gpulse(t_axis, loc) for loc in dlocs]), axis=0)


In [ ]:
plt.plot(t_axis, K, label='K')
plt.plot(t_axis, D, label='D')
plt.plot(t_axis, P, label='P')
plt.xlabel("time (s)")
plt.ylabel("envelope (V)")
plt.xlim([0, 1])
plt.legend()